### Kütüphanelerin Yüklenmesi

In [ ]:
## Proje içerisinde sıklıkla kullanılacak olan (genel kullanıma sahip) kütüphanelerin yüklenmesi

import numpy as np 
import pandas as pd 
import os
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import iplot

In [ ]:
## Projeye dosya olarak eklenmiş olan verilerin dosya isimlerinin gösterilmesi

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

### Verinin Yüklenmesi

In [ ]:
## .csv formatında olan verinin projeye eklenmesi

datemp = pd.read_csv("../input/daily-temperature-of-major-cities/city_temperature.csv")

### Verinin Tanınması & İncelenmesi

In [ ]:
# Veri tablosunun kolonlarının listelenmesi

datemp.columns

In [ ]:
# Top 10 data 

datemp.head(10)

In [ ]:
# Bottom 10 data 

datemp.tail(10)

In [ ]:
## Veri Tablosundan Rastgele 10 Değerin Alınması

datemp.sample(n=10)

In [ ]:
datemp.info()

In [ ]:
# All numeric columns has described by list

datemp.describe()

In [ ]:
# Describe of datemp columns' "AveTemperature"

datemp["AvgTemperature"].describe()

### Özellik-Nitelik Analizi (Feature Analysis):

#### Adım 1 : Eksik Verilerin Belirlenmesi (Missing Data Detection)
#### Adım 2 : Verilerin Birbirleriyle Olan İlişkilerinin İncelenmesi (with Correlation Map)
#### Adım 3 : Sınırdışı-Outlier Verilerin Belirlenmesi (Outlier Detection)

### 1- Eksik Verilerin Belirlenmesi (Missing Data Detection)

In [ ]:
# Eksik Verilerin Matris Formunda Gösterimi

msno.matrix(datemp)
plt.show()



In [ ]:
## Eksik Verilerin Histogram Formatında Gösterimi

msno.bar(datemp)
plt.show()

* Grafiğe bakıldığında sadece "State" kolonunda veri eksikliği mevcuttur. (2906327 - 1455337 = 1450990)

* Sadece "State" kolonunda eksikliğin olması diğer kolonlardaki veya "State" kolonundaki verilerin düzgün *(bir kolonun tüm verilerinin aynı veri tipinde, aynı formatta olması)* olması anlamına gelmemektedir. Kolonlardaki veri tipleri ve formatlar kontrol edilmelidir.


### Verilerin Birbirleriyle Olan İlişkilerinin İncelenmesi (with Correlation Map)

In [ ]:
## Verilerin Korelasyon Katsayılarından Faydalanılarak Heatmap Gösteriminin Çıkarılması

corr_values = datemp.corr()

f,ax = plt.subplots(figsize=(10,10))
sns.heatmap(corr_values,annot=True,linewidth=0.5,linecolor="White",fmt=".2f",ax=ax)
plt.show()

* Heatmap incelendiğinde (sayısal) kolonlar arasında ciddi bir doğru orantı(0,1) veya ters orantı(-1,0) olduğu görülmemektedir. Zaten kolonlardaki verilere bakıldığında bu sonuç beklenebilir çünkü tarih verilerinin arasında böyle orantılı bir değer çıkması mümkün görülmemektedir. 

### Sınırdışı-Outlier Verilerin Belirlenmesi (Outlier Detection)

In [ ]:
## Veride Yer Alan Bölge(Region) Bilgisine Göre (Çeşitli Parametrelerle Destekli) Ortalama Sıcaklıklar İçin Kutu Grafiğini (Box Plot) Oluşturan Fonksiyon


def RegionByTemperature(Region,TimeType,Category,data,limit=None,suptitle=None):

    """
    Returns seaborn box plot by region of data.

    Parameters:
        Region (str) : The string which is to be reversed.
        TimeType (int) : Month(1,2,...,12), Day(1,2,...31), Year(1995,1996,...,2013)
        Category (various) : The data has categorized by Year, Month, State, Country which in data columns.
        Limit (int) : The data gets by limit value.

    Returns:
        Plot(seaborn plot) : The Seaborn Box Plot    
    """
    
    import seaborn as sns
    
    if type(limit) == int:
        
        dataTemporary = data[data["Region"] == Region].iloc[0:limit]
        
    else:
        
        ## if limit has not int type it is equalled 1000.

        dataTemporary = data[data["Region"] == Region].iloc[0:1000]

    sns.boxplot(x='Region',y=TimeType,hue=Category,data = dataTemporary)
    plt.suptitle(suptitle)
    plt.show()
    

In [ ]:
## Region : Europe, Time Type : AvgTemperature, Category : Year

RegionByTemperature("Europe","AvgTemperature","City",datemp,limit=100000,suptitle="AvgTemperature by Region")


In [ ]:
datemp[datemp["City"] == "Nicosia"].sample(n=10)

* BoxPlot grafiğinin en önemli kullanım alanlarından biri Outlier(Sınırdışı-Ortalamanın çok dışında) değerlerin tespitidir. Burada görülmektedir ki "City" kolonunda yer alan "Nicosia" değerinde ortalamanın çok dışında değerler mevcuttur.(Outlier değerlerin tespitinde outlier hesapları kullanılmaktadır, ancak burada açık ara hangi verinin outlier olduğu bellidir.) Bu değerler verinin yorumlanmasında yanlışlıklara neden olabilir bu nedenle ya veriden çıkartılmalı ya da dikkate alınmamalıdır.

In [ ]:
## Region : North America, Time Type : AvgTemperature, Category : City

RegionByTemperature("North America","AvgTemperature","Year",datemp,limit =100000)

In [ ]:
datemp[datemp["City"] == "Guadalajara"].sample(n=10)

* "City" kolonunda yer alan "Guadalajara" değeri de outlier değerler içermektedir.

In [ ]:
## Şehirlere Göre Ortalama Sıcaklığı Veren Fonksiyon

def AverageValueByCountry(country,data=datemp):
    
    """
    Returns average temperature by country of data.

    Parameters:
        Country (str) : The string which is to br reversed.
        data(dataframe) : Data is used to calculate average temperature values.
    Returns:
        aveValue(float) : Average temperature value by country feature in data
        
    """
    
    aveValue = data[data["Country"]==country]["AvgTemperature"].mean()
    
    return aveValue


In [ ]:
AverageValueByCountry("Turkey")

In [ ]:
## Bölge Değeri 'Middle East' Olan Ülkelerin Ortalama Sıcaklıklarının Listesi

countries = datemp[datemp["Region"]=="Middle East"]["Country"].unique()
me_countries = {}

for c in countries:
    
    me_countries[c] = AverageValueByCountry(c)


me_countries

In [ ]:
## Belirlenen Bölge Değerine Göre Ülkelerin Ortalama Sıcaklıklarını Gösteren Pasta Grafiği (Pie Chart) 


dfMiddleEast = datemp[datemp["Region"]=="Middle East"]
df = pd.DataFrame({'Country': list(me_countries.keys()), 'Average Temperature': list(me_countries.values())})

pie1 = df["Average Temperature"]

labels = df["Country"]

piechart = {"values":pie1,
           "labels":labels,
           "type":"pie"}


data = [piechart]

layout={"title":"Average Temperature of Country in Middle East Region"}

fig = go.Figure(data=data,layout=layout)

iplot(fig)

In [ ]:
## Belirlenen Bölge Değerine Göre Ülkelerin Ortalama Sıcaklıklarını Gösteren Pasta Grafiği (Pie Chart) 
## Ortalama Sıcaklıkların Ülkeler Bazında Toplanmasıyla Oluşturulmuştur.

dfMiddleEast = datemp[datemp["Region"]=="Middle East"]

pie1 = datemp["AvgTemperature"]

labels = datemp[datemp["Region"]=="Middle East"]["Country"]

piechart = {"values":pie1,
           "labels":labels,
           "type":"pie"}


data = [piechart]

layout={"title":"Average Temperature of Country in Middle East Region"}

fig = go.Figure(data=data,layout=layout)

iplot(fig)

In [ ]:
dt_RegionTemperature = datemp.loc[:,["Region","AvgTemperature"]]

dt_RegionTemperature.groupby("Region").first()

* Region = Europe verisinin ortalaması olan -99.0 sayısı bir outlier veridir. Europe verisinden bu alan temizlenmeli ya da ortalamaya uygun bir değer verilmelidir.
* Örneğin lokasyon olarak Europe bölgesine yakın olan Middle East, Asia ve Africa bölgelerinin ortalaması alınabilir.



In [ ]:
ax = plt.subplots(figsize=(20,10))
sns.pointplot(x=dt_RegionTemperature["Region"],y=dt_RegionTemperature["AvgTemperature"])



plt.xlabel("Region",fontsize=15,color="red")
plt.ylabel("AvgTemperature",fontsize=15,color="red")

plt.title("Average Temperature by Region", fontsize=20,color="red")

plt.grid()
plt.show()

* Europe bölgesinin grafikte bu kadar aşağılarda görünmesi içerdiği outlier veriler yüzündendir.

In [ ]:
sns.distplot(dt_RegionTemperature['AvgTemperature'], kde=True)

In [ ]:
chart = sns.violinplot(dt_RegionTemperature["Region"],dt_RegionTemperature["AvgTemperature"])

chart.set_xticklabels(chart.get_xticklabels(), rotation=90)

plt.show()

### Regresyon

In [ ]:
datareg = datemp.copy()

In [ ]:
datareg.sample(n=25)

* AvgTemperature kolonundaki veriler bazı kolonlar kullanılarak tahmin edilecek. Gözetimli öğrenme kullanılarak yapılacak bu işlemde sayısal verilerin tahmini söz konusu olduğundan regresyon işlemi uygulanacaktır. Regresyon işlemi basit manasıyla koordinat sisteminde yerleştirilen veriler için bir eğri uydurma probleminin çözülmesidir. Burada bir veya birden fazla girdi(input)(x-values) değeri kullanılarak tek bir çıktı(output)(y-value) değeri mevcuttur. Bu durumda AvgTemperature kolonu çıktı değeri olarak değerlendirilecek.

In [ ]:
# AvgTemperature içerisinde bulunan outlier değerlerin temizlenmesi (ortalamaya uygun bir değerle değiştirilmesi)

avgNonOutlier = datareg[datareg["AvgTemperature"]>-90]["AvgTemperature"]

avgColumnMean = avgNonOutlier.mean()

datareg["AvgTemperature"] = datareg["AvgTemperature"].replace(-99.0,avgColumnMean) ## Outlier değerler ortalama değerlerle değiştirildi.

In [ ]:
datareg.sample(n=25)

* Veriye bakıldığında AvgTemperature kolonunun tahmin edileceği düşünüldüğünde bu kolonu etkileyecek, etkileme ihtimali olan kolonların regresyon işleminde birer girdi olarak düşünülmesi gerekmektedir.
* Region, Country, State, City, Month, Day, Year kolonları girdi-input-x values olarak değerlendirilebilir. Ancak daha önce yapılan gözlemlerde State kolonunun fazlaca NaN değere sahip olduğu göz önünde bulundurulduğunda State kolonunun alınmasına gerek yoktur.


### Data Eksiltme

In [ ]:
datareg = datareg.sample(n=5000)

In [ ]:
# Girdi - Input - X Values 

data_x = datareg.loc[:,["Region","Country","City","Month","Day","Year"]]

data_x.sample(n=15)

* Girdi değerleri incelendiğinde hem kategorik değerler hem de ay - yıl - gün gibi sayısal olsa bile sonuca sayısının niceliği ile etki etmeyecek değerler mevcut. Böyle değerlerin regresyonda girdi olarak kullanılabilmesi için encoder işlemine tabi tutulması gerekmekte. Çünkü hem değerler bu şekilde kategorik olarak kullanılamaz hem de bazı sayısal değerler sonucu şaşırtabilir. Mesela aydaki günün sayısı arttıkça sıcaklık artmaz veya azalmaz yani aydaki günün 5 ile 20 olması arasında sıcaklık anlamından bir olumlu veya olumsuz ilişki bulunmamakta.(Genel manada böyle düşünülebilir) Aynı durum diğer sayısal değerler için de geçerlidir. Kategorik veriler ise kategorisinin adı ile değil de birer vektör olarak ifade edildiğinde ancak regresyon işlemine tabi tutabilir. Çünkü regresyon işleminde fonksiyon(lar) kullanılır ve bir fonksiyon elma ile armutu meyve ile sebzeyi bilemez sayıları veya vektörler gibi nicel değerleri bilir.

In [ ]:
# Çıktı - Output - Y Value

data_y = datareg.loc[:,["AvgTemperature"]]

data_y.sample(n=15)

In [ ]:
## Girdi değerlerinin regresyona uygun hale getirilmesi : One Hot Encoder İşlemi

region_oht = pd.get_dummies(data_x["Region"])
country_oht = pd.get_dummies(data_x["Country"])
city_oht = pd.get_dummies(data_x["City"])
month_oht = pd.get_dummies(data_x["Month"]) 
day_oht = pd.get_dummies(data_x["Day"]) 
year_oht = pd.get_dummies(data_x["Year"])

In [ ]:
# Girdi değerlerine uygulanan One-Hot-Encoder işleminden sonra oluşan kolon sayıları

print("Region OHT Shape: ", region_oht.shape)
#print("Country OHT Shape: ",country_oht.shape)
#print("City OHT Shape: ",city_oht.shape)
print("Mounth OHT Shape: ",month_oht.shape)
print("Day OHT Shape: ",day_oht.shape)
print("Year OHT Shape: ",year_oht.shape)

* Veri tablosunda yer alan kolonlardan city kolonu içerisinde 321 adet kolon "özellik-nitelik" olarak yeni veri tablosuna eklenecektir. "city" kolonu regresyon hesaplamaları sırasında tekrardan değerlendirilebilir. Şimdilik pas geçilecektir. Aynı şekilde "day" kolonu da regresyon hesaplamalarının dışında tutulacaktır.

In [ ]:

#data_x_oht = pd.concat([region_oht,country_oht,city_oht,day_oht,year_oht,month_oht],axis=1)


data_x_oht = pd.concat([region_oht,day_oht,month_oht,year_oht],axis=1)

data_x_oht.sample(n=15)

* Tüm girdi kolonlarına One-Hot-Encoder işlemi uygulandı daha önce 6 olan kolon sayısı şimdi 525 tane oldu. Yani 1 adet sıcaklık değerinin tahmin edilmesi için 525 tane değerin ayrı ayrı hesaplanması gerekmekte. Daha da ilginç olanı y = ax+b gibi bir denklemde bir x değeri ve bir de y değeri mevcuttur. Böyle bir denklem için iki boyutlu bir uzay gösterim için yeterlidir. Ancak 525 kolon kullanılarak tahmin edilen 1 adet sıcaklık değeri denklemi için 526(525+1) boyutlu bir uzay gerekmektedir. 

* Elbette sınırsız kaynaklarda böyle bir problem için en iyisi 525 tane kolonun da aynı anda kullanılması güzel bir yöntem olabilirdi. Fakat sınırsız kaynaklar mevcut olmadığı için 525 tane kolonun mantıklı bir şekilde azaltılması gerekmektedir.

* Bazı niteliklere uygulanan one-hot-encoder işlemi sonrasında çok fazla kolon ortaya çıkmaktadır. Örneğin "day" kolonu için 30 farklı kolon veri tablosuna eklenmektedir. Bu nedenle bazı kolonlar(nitelikler,feature'lar) ya hiç alınmayacak ya  bilinçli şekilde azaltılacak ya da içerisindeki veriler sınıflara bölünecektir. 

In [ ]:
# Oluşturulan veri setinin kolonları kontrol edilir
data_x_oht.columns

* data_x_oht veri setine bakıldığında kolonların bazılarının anlamsız olduğu gözlemlenmektedir. (201,200 kolonlarının ne tür bir anlam içerdiği meçhuldur. Yıl, Ay, Gün veya bir Bölge adı olmadığı aşikardır.) Bu nedenle anlamsız olan kolonların data_x_oht veri setinden kaldırılması veri temizliği açısından elzemdir.

In [ ]:
print("Input Shape: ",data_x.shape)
print("Input Shape with OneHotEncoder: ",data_x_oht.shape)
print("Output Shape: ",data_y.shape)

In [ ]:

from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(data_x_oht,data_y,test_size=0.33)

## train_test_split fonksiyonu data_x_oht ve data_y veri tablolarını karıştırır ve train-test olarak iki tipe böler. train tipindeki veriler modeli kurmakta kullanılacakken test tipinde ayrılanlar modelin başarısını test etmek için kullanılacaktır.


In [ ]:
print("x_train Shape ->",x_train.shape)
print("x_test Shape ->",x_test.shape)
print("y_train Shape ->",y_train.shape)
print("y_test Shape ->",y_test.shape)

In [ ]:
x_train.head()

In [ ]:
x_test.head()

In [ ]:
y_train.head()

In [ ]:
y_test.head()

### Doğrusal Regresyon ~ Linear Regression

In [ ]:

from sklearn.linear_model import LinearRegression

lreg = LinearRegression()

lreg.fit(x_train,y_train)
    
lreg.score(x_test,y_test)



In [ ]:
def OLS_Table(data_x_oht, data_y):

    import statsmodels.api as sm
    import numpy
    
    cumdata = np.append(arr=np.ones((data_x_oht.shape[0],1)).astype(int),values=data_x_oht,axis=1)
    
    r = sm.OLS(endog=data_y,exog=data_x_oht)
    
    r_ols = r.fit()
    
    return r_ols.summary()
    

In [ ]:
OLS_Table(data_x_oht,data_y)

* OLS Tablosuna göre bazı kolonların elenmesi gerekmektedir. Çünkü ilgili kolonların modele katkı sağlamasından ziyade yük bindirdiği farkedilmektedir. OLS tablosuna göre hangi kolonların eleneceği hesabı yapılırken kolonların aslında p-value değerlerine bakılmaktadır. p-value değerinin hesabının yapılması demek ise "Hipotez Testi" yapılması demektir.

* Kolonlar çıkartılırken P>|t| kolonuna bakılır eğer bu kolon 0.5 (genelde p-value için kabul edilen değer 0.05 dir) değerinden büyük ise ilgili kolon girdi değerleri arasından çıkartılmalıdır.

* <b>Çıkartılması gereken kolonlar :</b>  Austria,  Bulgaria, Costa Rica, Croatia, France, Georgia, Guinea-Bissau, Haiti, Hungary, Israel, Kenya, Lebanon, Mozambique, Namibia, Peru, Romania, Sierra Leone, Slovakia, Switzerland, The Netherlands, Turkmenistan, Ukraine, United Kingdom, Zambia

In [ ]:
## Çıkartılacak olan kolonlar ayrılırlar:

string = "Austria,Algeria,Bulgaria,Brazil,Costa Rica,Croatia,Denmark,\
France,Georgia,Guinea-Bissau,Gambia,Georgia,Guatemala,Haiti,Hungary,Honduras,Israel,Ireland,Kenya,Lebanon,\
Mozambique,Mauritania,Morocco,Namibia,Peru,Romania,Sierra Leone,Slovakia,Syria,Switzerland,\
The Netherlands,Turkmenistan,Ukraine,United Kingdom,Tunisia,Yugoslavia,Zambia"

ex_columns = [i.strip() for i in string.split(",")]


In [ ]:
print("Düzenleme işlemi yapılmadan önce kolon sayısı : ",len(data_x_oht.columns))

data_x_fit = pd.DataFrame()

for i in data_x_oht.columns:
    
    if i not in ex_columns:
        data_x_fit = pd.concat([data_x_fit,data_x_oht[i]],axis=1)

print("Düzenleme işlemi yapıldıktan sonra kolon sayısı : ",len(data_x_fit.columns))

data_x_fit.head()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data_x_fit,data_y,test_size=0.30)

print("X_Train : ",len(x_train))
print("X_Test : ",len(x_test))
print("Y_Train : ",len(y_train))
print("Y_Test : ",len(y_test))

from sklearn.linear_model import LinearRegression

lreg = LinearRegression()

lreg.fit(x_train,y_train)

lreg.score(x_test,y_test)

In [ ]:
from sklearn.metrics import r2_score

r2Score_lreg = r2_score(data_y,lreg.predict(data_x_fit))

r2Score_lreg

### Polinom Regresyon ~ Polynomial Regression

In [ ]:
 # Degree = 2

from sklearn.preprocessing import PolynomialFeatures

poly_2 = PolynomialFeatures(degree=2)

x_poly_2 = poly_2.fit_transform(data_x_oht)

lr_model_2 = LinearRegression()

lr_model_2.fit(x_poly_2,data_y)

lr_model_2.predict(poly_2.fit_transform(data_x_oht))

## Score 

from sklearn.metrics import r2_score

r2Score_lr2 = r2_score(data_y,lr_model_2.predict(x_poly_2))

r2Score_lr2

In [ ]:
 # Degree = 3

from sklearn.preprocessing import PolynomialFeatures

poly_3 = PolynomialFeatures(degree=3)

x_poly_3 = poly_3.fit_transform(data_x_oht)

lr_model_3 = LinearRegression()

lr_model_3.fit(x_poly_3,data_y)

lr_model_3.predict(poly_3.fit_transform(data_x_oht))

## Score 

from sklearn.metrics import r2_score

r2Score_lr3 = r2_score(data_y,lr_model_3.predict(x_poly_3))

r2Score_lr3

### Destek Vektör Regresyonu ~ Support Vector Regression

In [ ]:
from sklearn.preprocessing import StandardScaler

sc1 = StandardScaler()

x_olcekli = sc1.fit_transform(data_x_oht.values.reshape(-1,1))
y_olcekli = sc1.fit_transform(data_y.values.reshape(-1,1))


In [ ]:
## Dataframe içerisinde yer alan kolonları tek tek standardize etmek gereklidir. Toplu halde standardize edilmek istendiğinde boyut farklılıkları oluşmaktadır. 
## Çünkü tüm dataframe yapısına reshape işlemi uygulandığında tüm kolonlar uç uca eklenmektedir.
    
print(data_x_oht["Middle East"].shape)
print(data_x_oht["Middle East"].values.reshape(-1,1).shape)
print(data_x_oht.shape)
print(data_x_oht.values.reshape(-1,1).shape)

## Burada data_x_oht veri setinin kolonlarının tek tek standardize edilmesi tercih edilebilir.


In [ ]:
from sklearn.svm import SVR

svr_reg = SVR(kernel="rbf") # kernel fonksiyon "rbf" olarak ayarlandı.

svr_reg.fit(data_x_oht,data_y)

svr_reg.predict(data_x_oht)

## Score 

from sklearn.metrics import r2_score

r2Score_svrreg = r2_score(data_y,svr_reg.predict(data_x_oht))

r2Score_svrreg

### Rassal Orman Karar Ağacı Regresyonu ~ Random Forest Decision Tree Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor


model_rf = RandomForestRegressor(n_estimators=11,random_state=0) # n_estimators : Kaç tane karar ağacı çizileceğinin sayısıdır.

model_rf.fit(x_train,y_train)

model_rf.predict(x_test)

## Score 

from sklearn.metrics import r2_score

r2Score_rfreg = r2_score(y_test,model_rf.predict(x_test))

r2Score_rfreg